# Imports
For this part of the hands on session, a few python package are required but they all can be installed simply using the command pip install

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf 
import lmfit # source: https://lmfit.github.io/lmfit-py/
from lmfit import minimize, Parameters, fit_report
from astropy.timeseries import LombScargle

# Hands on session Part II

This part of the lbl hands on session is about to play with a toy model to better understand the data produced by lbl and what is embebded in them.

Please note that this is a toy model so it is not represent the true lbl computations.

## Build a line template 

Every lbl computation are made compared to a template so let's build the template of one line. 

For this toy model, every line will be considered as gaussian lines (or skewed gaussian if you want to try a more complex model with assymetric lines) build using the following function:

In [2]:
def Gauss(x, a, sigma):
        return 1 - a * np.exp(-x**2 / (2 * sigma**2))


def skewed_gaussian(x, a, sigma, alpha) :
    def cdf(x) :
        return((1/2)*(1+erf(x/np.sqrt(2))))

    return 1 + (2/sigma)*(-a/2)*np.exp(-(x)**2 / (2 * sigma**2))*cdf(alpha*(x)/sigma)

In [ ]:
a0 =        #set your line depth
sigma_0 =   #set your line width
x_0 =       #set your line x-shift 

x = np.arange(-2,2,1/10) 
Template = Gauss(x-x_0, a0, sigma_0)

plt.plot(x, Template, "g")
plt.show()

## Build a line observation

So here is your template for that line. 

Then build one observation of that line by building another line with slightly different paramaters (for consistent results please do not exceed a few percent of the initial template parameter)

In [ ]:
a_l =          #set your line depth
sigma_l =      #set your line width
x_l =          #set your line x-shift

line = Gauss(x+x_l, a_l, sigma_l)

plt.plot(x, Template, "g")
plt.plot(x, line)
plt.show()

#lets plot the difference between the line and the template
plt.plot(x, line-Template)
plt.show()

## Compute the bouchy equation

lbl is using the Bouchy's framework (Bouchy et al. 2001): to simplify, the difference between the line observed $A$ and the template $A_0$ can be decomposed into a Taylor serie as following :

$A(i)-A_0(i) = d_0v + dv \frac{d A_0(i)}{d \lambda(i)} + d_2v \frac{d^2 A_0(i)}{d \lambda(i)^2} + d_3v \frac{d^3 A_0(i)}{d \lambda(i)^3} + \mathcal{O}(\frac{d^3 A_0(i)}{d \lambda(i)^3}) $

What lbl do is to compute $d_0v$, $dv$, $d_2v$, $d_3v$ ... In our model we will fit those parameters. 

Please note that this is a toy model so it is not represent the true lbl computations. For more details, please refer to Bouchy+01 and Artigau+22 or dig into the lbl code to see how it is truely computed. 

In [4]:
#bouchy equation to minimize 
def equation(params, d1, d2, d3, diff):
    d0v = params.get('d0v').value
    dv  = params.get('dv').value
    d2v = params.get('d2v').value
    d3v = params.get('d3v').value
    z = d0v + dv*d1 + d2v*d2 + d3v*d3 - diff
    return(z)

def fit_bouchy(line, Template): 

    #get difference between line and template
    diff_seg = line - Template
    
    #get Template Gradient
    d1 = np.gradient(Template, x) #first dervative
    d2 = np.gradient(d1, x)       #second dervative
    d3 = np.gradient(d2, x)       #second dervative
    
    #initialize
    params_ini = Parameters()
    params_ini.add('d0v', value=0, min=-np.inf, max=np.inf)
    params_ini.add('dv', value=0, min=-np.inf, max=np.inf)
    params_ini.add('d2v', value=0, min=-np.inf, max=np.inf)
    params_ini.add('d3v', value=0, min=-np.inf, max=np.inf)
    #fit
    out = minimize(equation, params_ini, args = (d1, d2, d3, diff_seg))
    
    return (out.params)

In [ ]:
result = fit_bouchy(line, Template)
print(result)

plt.plot(x, equation(result, np.gradient(Template, x), np.gradient(np.gradient(Template, x), x), np.gradient(np.gradient(np.gradient(Template, x), x), x), line-Template)+line-Template, label='fit')
plt.plot(x, line-Template, label='A(i)-A0(i)')
plt.legend()
plt.show()

plt.plot(x, equation(result, np.gradient(Template, x), np.gradient(np.gradient(Template, x), x), np.gradient(np.gradient(np.gradient(Template, x), x), x), line-Template)+line, label='fit')
plt.plot(x, Template, label='Template')
plt.plot(x, line, label='Line')
plt.legend()
plt.show()

# Playground ! 

Now, up to you to do wathever tests you want and see how lbl behaves. There is no limit but your imagination. 

If you don't know what to do first, here are suggestions of physically motivated tests that you can try:

- Modulate the gaussian parameters within simulated observation. 
    - Modulate the line depth 'a' could simulated temperature variations
    - Modulate the x-offset would simulate a radial velocity doppler-shift
    - Modulate the line width \sigma could simulate a Zeeman broadenening induced by the magnetic field for example 
        - If you are working with multiple lines. You can attribute a different "sensitivity" for each lines with respect to that modulation. It could emulated Landé g factors. 
    - Use skewed to emulate BIS. 
- Add photon noise
    - On a single line. 
    - On multiple lines 
        - LBL is known to be resilient to noise by averaging the data among all the lines. Thats how we obtain High precision RVs for example. (to be precise, inside lbl we use the odd_ratio_mean() function to average over all the lines. This function can be found in lbl/lbl/core/math.py file of your lbl installation. So you can import it using `from lbl.core import math as mp` and then call it with `mp.odd_ratio_mean()`). 
        - SPIRou lines are known to be more noisy in the bluest part of the spectrum
- Build lbl data cube 
    1. Build a multi line template with different parameters to emulate a template sprectum. 
    2. Build any observation you want by modulating whatever parameter along time. 
    3. Add noise (optional)
    4. Run the fit (depending on how big your data cube is you might need to parallelize your code for computation time saving)
    5. Exploit your so produced lbl data !
        - by averaging them (odd_ratio_mean())
        - by applying a PCA
        - by fitting GPs or Keplerian model to the averaged/to the PCA component
        
Feel free to edit every existing function that I've provided if it's more convenient to you